<a href="https://colab.research.google.com/github/guoweihua0307/guoweihua0307/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install requests
!pip install python-docx

import requests
from bs4 import BeautifulSoup
import os
from docx import Document
from docx.shared import Inches

# 定义保存数据的列表
data = []

# 定义搜索的关键词
keyword = '讣告'

# 发送搜索请求并获取网页数据
url = f'https://www.obituare.com/'  # 替换为实际的搜索网站URL
response = requests.get(url)
html = response.text

# 解析网页数据
soup = BeautifulSoup(html, 'lxml')
items = soup.find_all('div', class_='article')  # 根据实际的HTML结构调整解析方式
for item in items:
    # 获取所需的数据并格式化
    post_date = item.find('span', class_='post-meta').text.strip()
    name = item.find('h1').text.strip()
    gender = '男' if item.find('img', alt='先生').parent.text.strip() == '先生' else '女'
    birth_date = item.find('p', class_='birth').text.strip()
    death_date = item.find('p', class_='death').text.strip()
    funeral_date = item.find('p', class_='funeral').text.strip()
    story = ' '.join(item.find('p', class_='story').text.split())
    achievements = ' '.join(item.find('p', class_='achievements').text.split())
    evaluation = ' '.join(item.find('p', class_='evaluation').text.split())
    relatives = ' '.join(item.find('p', class_='relatives').text.split())
    publisher = item.find('p', class_='source').text.strip()
    data_dict = {
        'post_date': post_date,
        'name': name,
        'gender': gender,
        'birth_date': birth_date,
        'death_date': death_date,
        'funeral_date': funeral_date,
        'story': story,
        'achievements': achievements,
        'evaluation': evaluation,
        'relatives': relatives,
        'publisher': publisher,
    }
    data.append(data_dict)

# 按照发布日期降序排列数据
data = sorted(data, key=lambda x: x['post_date'], reverse=True)

# 创建doc文件并写入数据
doc = Document()
for item in data:
    table = doc.add_table(rows=1, cols=len(item))
    for col in range(len(item)):
        table.cell(0, col).text = item[col]
    doc.add_page_break()  # 根据需要添加分页符

# 将文件保存到本地
file_name = f'obituaries_{keyword}.docx'  # 根据需要修改文件名和路径
doc.save(file_name)

